In [3]:
import numpy as np
from collections import Counter
import random
import pandas

In [4]:
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 
 'g', 'h', 'i', 'j', 'k', 'l', 
 'm', 'n', 'o', 'p', 'q', 'r', 
 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

freqs = {i:0 for i in alphabet}

In [5]:
sample = [alphabet[int(random.randint(0,len(alphabet)-1))] for i in range(16)]

In [6]:
def vectorize_counts(sentence):
        alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 
                'g', 'h', 'i', 'j', 'k', 'l', 
                'm', 'n', 'o', 'p', 'q', 'r', 
                's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
        freqs = {i:0 for i in alphabet}
        
        for i in sentence:
                
            if i in alphabet:
                freqs[i] += 1
                
        return list(freqs.values())

In [7]:
vectorize_counts(str("".join(sample)))

[2, 1, 0, 1, 1, 0, 0, 0, 0, 1, 2, 2, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 2, 0, 0, 0]

In [8]:
with open('../src/data/Author: William Shakespeare.txt', 'r') as file:
    data = file.readlines()

In [22]:
res = []

for i in data:
    temp = vectorize_counts(i)
    res.append(temp)

bag_of_words = np.array(res).reshape(len(data),26)

In [25]:
maxlen = 16
sequences = []
for doc in bag_of_words:
    for i in range(len(doc)-maxlen):
        seq = doc[i:i+maxlen]
        target = doc[i+maxlen:]
        sequences.append((seq, target))
sequences

[(array([1, 1, 1, 0, 2, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0]), 0),
 (array([1, 1, 0, 2, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0]), 0),
 (array([1, 0, 2, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]), 0),
 (array([0, 2, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]), 2),
 (array([2, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2]), 0),
 (array([0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0]), 0),
 (array([0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0]), 0),
 (array([1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0]), 0),
 (array([1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0]), 0),
 (array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0]), 0),
 (array([3, 0, 0, 0, 3, 0, 0, 2, 2, 0, 1, 2, 1, 0, 1, 1]), 0),
 (array([0, 0, 0, 3, 0, 0, 2, 2, 0, 1, 2, 1, 0, 1, 1, 0]), 2),
 (array([0, 0, 3, 0, 0, 2, 2, 0, 1, 2, 1, 0, 1, 1, 0, 2]), 1),
 (array([0, 3, 0, 0, 2, 2, 0, 1, 2, 1, 0, 1, 1, 0, 2, 1]), 1),
 (array([3, 0, 0, 2, 2, 0, 1, 2, 1, 0, 1, 1, 0, 2, 1, 1]), 1),
 (array([0, 0, 2, 2, 0, 1, 2, 1, 0, 1, 1, 0, 2, 1, 1, 1

In [24]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.utils import *

vocabulary = alphabet
maxlen = 26
num_epochs = 15

# create the model
model = Sequential()
model.add(Embedding(len(vocabulary), 100, input_length=maxlen))
model.add(LSTM(100))
model.add(Dense(len(vocabulary), activation='softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

# train the model
for epoch in range(num_epochs):
    for seq, target in sequences:
        x = np.array([seq])
        y = to_categorical(target, num_classes=len(vocabulary))
        model.train_on_batch(x, y)

ValueError: Data cardinality is ambiguous:
  x sizes: 1
  y sizes: 26
Make sure all arrays contain the same number of samples.